In [ ]:
import sys
sys.path.insert(1, 'utils')
from utils import *
from data_manipulation import *
pd.options.display.max_colwidth = 100

# Categorías

In [ ]:
getCategories()

### Para la selección de categorías:

#### La idea de usar solo el búscador de categorías para armar el dataset, es porque el buscador por query puede traer resultados que estan relacionados al producto, pero no lo son, en cambio el búscador por categorías es bastante preciso en los productos que devuelve.
#### Dado que no estoy usando un acess token para traer mas de 1000 resultados por categoría, decidí que la mejor manera de capitalizar la cantidad de productos disponibles era usar el búscador de categorías.

## Categorías elegidas para el estudio
* MLA1002 (televisores)
* MLA1055 (celulares y smartphones)
* MLA398582 (Heladeras)
* MLA1644 (Aires Acondicionados)
* MLA438566 (Consolas)
* MLA1652 (Notebooks)

#### Las categorías elegidas representan, aquellas que según mi criterio son muy populares por los usuarios de MeLi

In [ ]:
seleted_categories = {
    'MLA1002': 'televisores', 
    'MLA1055': 'celulares',
    'MLA398582': 'heladeras',
    'MLA1644': 'aires',
    'MLA438566': 'consolas',
    'MLA1652': 'notebooks'
}

In [ ]:
categories_products = pd.DataFrame(columns=['id', 'site_id', 'title', 'seller', 'price', 'prices', 'sale_price',
       'currency_id', 'available_quantity', 'sold_quantity', 'buying_mode',
       'listing_type_id', 'stop_time', 'condition', 'permalink', 'thumbnail',
       'thumbnail_id', 'accepts_mercadopago', 'installments', 'address',
       'shipping', 'seller_address', 'attributes', 'differential_pricing',
       'original_price', 'category_id', 'official_store_id', 'domain_id',
       'catalog_product_id', 'tags', 'catalog_listing', 'order_backend'])

### Extraigo publicaciones de las categorías elegidas

In [ ]:
for cat, cat_name in seleted_categories.items():
    sub = searchItemsByCategory(cat)
    print("Bajando publicaciones de: {}-{}".format(cat,cat_name))
    sub['category'] = cat_name
    print("Resultados: {}".format(sub.id.count()))
    categories_products = pd.concat([categories_products, sub], sort=False)

### Extraigo el detalle de cada publicación porque tienen mas features que el buscador general o mas detalles de cada una

In [ ]:
items = searchItemsDetail(categories_products)

### Guardado de dataframes (correr solo si no existen los archivos)

In [ ]:
items.to_csv("data/items.csv", index=False)
categories_products.to_csv("data/categories_products.csv", index=False)

### Cargado si ya existen (correr si existen los archivos)

In [ ]:
#categories_products = pd.read_csv("data/categories_products.csv")
#items = pd.read_csv("data/items.csv")

### Combino ambos sets

In [ ]:
results = pd.merge(categories_products, items, left_on='id', right_on='id', how="left", suffixes=('', '_y'), )
results = results.filter(regex=r'.*(?<!_y)$')

### Algunas columnas que me parecieron interesantes tenerlas para el análisis o para la generación de features

In [ ]:
results['discount_per'] = results.apply(lambda x: 0 if np.isnan(x.original_price) else int(100 - (x.price / x.original_price)*100),
                                               axis=1) ## Porcentaje de descuento
results['installment_num'] = results.apply(lambda x: get_installments_data(x, 'quantity'), axis=1) ## Cantidad de cuotas
results['installment_rate'] = results.apply(lambda x: get_installments_data(x, 'rate'), axis=1) ## Interés de las cuotas
results['brand'] = results.apply(lambda x: get_brand(x), axis=1) ## Marca
results['seller_status'] = results.seller.apply(lambda x: get_seller_rep(x)) ## Rating del seller
results['pic_qty'] = results.pictures.apply(lambda x: get_picture_qty(x)) ## Cantidad de imagenes
results['has_video'] = results.video_id.apply(lambda x: not (x is None)).astype('int') ## Tiene video
results['warrant_time'] = results.apply(lambda x: get_warrant_time(x), axis=1) ## Meses de garantía
results['is_official'] = results.official_store_id.apply(lambda x: math.isnan(x)).astype('int') ##Es tienda oficial 
results['has_free_shipping'] = results.shipping.apply(lambda x: x['free_shipping']).astype('int') ## Tiene envío gratuito
results['qty_attributes'] = results.attributes.apply(len) ## Cuantos atributos hay cargados, este era el mas cross para múltiples categorías
results['qty_variations'] = results.variations.apply(lambda x: variations_qty(x)) ## Cuantas variaciones hay de cada publicación

results = results.astype({"sold_quantity": int, "available_quantity": int})
results = results.astype({"date_created": "datetime64"})
results = results.astype({"last_updated": "datetime64"})

results['created_date_diff'] = results.date_created.apply(lambda x: (datetime.now() - x).days) ## Antiguedad en días de la publicación
results['created_date_diff_decile'] = pd.qcut(results.created_date_diff, 10, labels=np.arange(10, 0, -1)) ## Antiguedad dividida en deciles
results['last_update_date_diff'] = results.date_created.apply(lambda x: (datetime.now() - x).days) ## Antiguedad desde último update a hoy
results['last_update_diff_decile'] = pd.qcut(results.last_update_date_diff, 10, labels=np.arange(10, 0, -1)) ## Antiguedad del último update en deciles
results['price_decile'] = pd.qcut(results.price, 10, labels=np.arange(10, 0, -1)) ## Precio en deciles

In [ ]:
results.to_csv('data/training_data.csv', index=False, na_rep='NULL') ## Guardo para usarlo en la generación del modelo